In [1]:
import json
import time
import re
from api_request.gpt35_turbo_completion import gpt35_turbo_completion


ontology = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book time: 13:30, 17:11, etc.
restaurant-book day: wednesday, friday, etc.
restaurant-book people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc.
"""

ontology = """hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book time: 13:30, 17:11, etc.
restaurant-book day: wednesday, friday, etc.
restaurant-book people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc."""

slot_description_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system.
The following lists are domain-slots and their descriptions.

domain-slots : description
hotel-name : name of the hotel
hotel-pricerange : price budget of the hotel
hotel-type : what is the type of the hotel
hotel-parking : parking facility at the hotel 
hotel-book stay : length of stay at the hotel
hotel-book day : day of the hotel booking
hotel-book people : number of people for the hotel booking
hotel-area : area or place of the hotel
hotel-stars : star rating of the hotel
hotel-internet : internet option at the hotel
train-destination : destination of the train
train-departure : departure location of the train
train-day : day of the train
train-book people : number of people booking for train
train-leaveat : leaving time for the train
train-arriveby : arrival time of the train
attraction-name : name of the attraction
attraction-area : area or place of the attraction
attraction-type : type of the attraction
restaurant-name : name of the restaurant
restaurant-food : food type for the restaurant 
restaurant-pricerange : price budget for the restaurant 
restaurant-area : area or place of the restaurant 
restaurant-book time : time of the restaurant booking 
restaurant-book day : day of the restaurant booking 
restaurant-book people : number of people booking the restaurant 
taxi-destination : destination of taxi 
taxi-departure : departure location of taxi
taxi-leaveat : leaving time of taxi 
taxi-arriveby : arrival time of taxi
"""

def parse_slot_with_bracket_matching(pred):

    # find all values where they are in the brackets

    # fix for no states
    if pred == "":
        return {}

    pred_slots = []

    # slot_value = pred.split(",")
    slots = re.findall(r'\((.*?)\)', pred)

    for i in slots:
        i = i.replace("(","").replace(")","")
        # for multiple slots in one bracket
        pred_slots.append(i)
    return pred_slots

In [12]:
# running_log_path = "expts/230614_1314-gpt35_turbo_5p_v2_sep_entity_sim_0.1_9_0to737/running_log.json"
running_log_path = "expts/230614_2133-gpt35_turbo_5p_v2_with_ett_0to100/running_log.json"

with open(running_log_path,'r') as f:
    content = json.load(f)
    
# for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong" and i["predicted_slot_values"] != i["turn_slot_values"]:
#         last_slot_values = {s: v.split('|')[0] for s, v in i['last_slot_values'].items()}
#         if not all(item in last_slot_values.items() for item in i["predicted_slot_values"].items()):
#             print("********** " + str(idx) + " **********")
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print(i["prompt"].replace(ontology,""))
#     #             print(i["prompt"])
#             print()
#             print("completion ==== " + i["completion"])
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n\n\n\n")

for idx, i in enumerate(content[:]):
    print("********** " + str(idx) + " **********")
#     print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#     print("user uttr  ==== " + i["dialog"]["usr"][-1])
#     print()
#     print("prev_label ==== " + str(i["last_slot_values"]))
    print()
    print(i["prompt"].replace(ontology,""))
    print()
#     print("ett_predict ==== " + str(i["ett_predicted_slot_values"]))
    print()
    print("completion ====" + i['completion'])
    print("prediction ==== " + str(i["predicted_slot_values"]))
    print("labeling.. ==== " + str(i["turn_slot_values"]))
    print("pred_status ==== " + str(i["pred_status"]))
    print("\n\n\n\n\n\n\n\n\n")

********** 0 **********

Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book_number_of_days: 1, 2, 3, etc.
hotel-book_day: monday, tuesday, etc.
hotel-book_people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book_people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:5

In [11]:
running_log_path = "expts/230608_1117-gpt35_turbo_5p_v2_0to737/running_log.json"
with open(running_log_path,'r') as f:
    content = json.load(f)
    
# for idx, i in enumerate(content):
#     if i["pred_status"] == "wrong" and i["predicted_slot_values"] != i["turn_slot_values"]:
#         last_slot_values = {s: v.split('|')[0] for s, v in i['last_slot_values'].items()}
#         if not all(item in last_slot_values.items() for item in i["predicted_slot_values"].items()):
#             print("********** " + str(idx) + " **********")
#             print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#             print("user uttr  ==== " + i["dialog"]["usr"][-1])
#             print()
#             print("prev_label ==== " + str(i["last_slot_values"]))
#             print()
#             print(i["prompt"].replace(ontology,""))
#     #             print(i["prompt"])
#             print()
#             print("completion ==== " + i["completion"])
#             print()
#             print("prediction ==== " + str(i["predicted_slot_values"]))
#             print("labeling.. ==== " + str(i["turn_slot_values"]))
#             print("\n\n\n\n\n\n\n\n\n")

for idx, i in enumerate(content[:200]):
    print("********** " + str(idx) + " **********")
#     print("sys  uttr  ==== " + i["dialog"]["sys"][-1])
#     print("user uttr  ==== " + i["dialog"]["usr"][-1])
#     print()
#     print("prev_label ==== " + str(i["last_slot_values"]))
    print()
    print(i["prompt"].replace(ontology,""))
    print()
#     print("ett_predict ==== " + str(i["ett_predicted_slot_values"]))
    print()
    print("completion ====" + i['completion'])
    print("prediction ==== " + str(i["predicted_slot_values"]))
    print("labeling.. ==== " + str(i["turn_slot_values"]))
    print("pred_status ====" + str(i["pred_status"]))
    print("\n\n\n\n\n\n\n\n\n")

********** 0 **********

Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.



Example #1
[context] (hotel-internet = yes), (hotel-type = guest house), (attraction-type = park), (hotel-parking = yes)
[system] sure ! milton country park is at cb46az . it s free admission .
[user] free is perfect . that will leave me extra to book a taxi from my hotel .
A: (taxi-departure = acorn guest house)


Example #2
[context] (taxi-destination = pembroke college)
[system] where will you be departing from ?
[user] i need to leave from primavera
A: (taxi-departure = primavera)


Example #3
[context] 
[system] 
[user] i want to get a taxi to pick me up from the cambridge train station please .
A: (taxi-departure = cambridge train station)


Example #4
[context] (restaura